In [1]:
import pandas as pd

In [24]:
def jaccard_similarity(row):
    string1, string2 = row.st_orig, row.st_pred
    
    wordset = lambda x: set(x.lower().split())
    a, b = wordset(string1), wordset(string2)

    c = a.intersection(b)

    return float(len(c)) / (len(a) + len(b) - len(c))

def string_length(s):
    return len(s)

def num_words(s):
    return len(s.split())

In [40]:
train = pd.read_csv("../data/train.csv")
submission = pd.read_csv("../roberta/submission_2020.05.10.00.16.csv")
merged = (train
          .merge(submission, on="textID")
          .rename({"selected_text_x": "st_orig", "selected_text_y": "st_pred"}, axis="columns")
          .assign(
              text=lambda x: x.text.str.lower(),
              st_orig=lambda x: x.st_orig.str.lower(),
              st_pred=lambda x: x.st_pred.str.lower(),
              jaccard=lambda x: x.apply(jaccard_similarity, axis=1),
              text_len=lambda x: x.text.apply(string_length),
              st_orig_len=lambda x: x.st_orig.apply(string_length),
              st_pred_len=lambda x: x.st_pred.apply(string_length),
              text_words=lambda x: x.text.apply(num_words),
              st_orig_words=lambda x: x.st_orig.apply(num_words),
              st_pred_words=lambda x: x.st_pred.apply(num_words),
          )
         )

In [42]:
merged.to_csv("train_merged.csv", index=False)